## Installation

Note : we are pinning the inspect_ai due to a recent breaking change

In [ ]:
%pip install -q openai anthropic ipywidgets colorama
import os
os.environ['XDG_RUNTIME_DIR']="/tmp"
os.environ['INSPECT_EVAL_MODEL'] = "openai/gpt-4o-mini"

from helpers.reporter.pretty import pretty_results

## Adding MCP stdio

# these ar installed in the docker image
#%pip install -q mcp mcp-server-tree-sitter mcp-server-git

In [ ]:
from inspect_ai.tool import mcp_server_sandbox, mcp_server_stdio
from inspect_ai import Task, eval, task

from inspect_ai.scorer import includes
from inspect_ai.dataset import Sample
from inspect_ai.agent import react

@task
def basic_mcp():
    patrick_scanner = mcp_server_stdio(
        command="python3",
        args=["./tools/patrick_scanner.py"]
    )

    patrick_fixer = mcp_server_stdio(
        command="python3",
        args=["./tools/patrick_fixer.py"]
    )

    git_server = mcp_server_stdio(
        command="python3",
        args=["-m", "mcp_server_git", "--repository", "/my-repo"]
    )

    treesitter_server = mcp_server_stdio(
            command="python3",
            args=["-m","mcp_server_tree_sitter.server" ]
        )

    treesitter_server_sandbox = mcp_server_sandbox(
         command="python3",
            args=["-m","mcp_server_tree_sitter.server" ],
      #      sandbox="mcp"
    )

    fs_server = mcp_server_sandbox(
       # sandbox="mcp",
        command="mcp-server-filesystem",
        args=["/"]
    )


#    fetch_server = mcp_server_stdio(command="python", args=["-m", "mcp_server_fetch"])

    return Task(
        dataset=[Sample(
            "What can you tell me about the current project in /my-repo? And check for syntax errors",
        )],
        sandbox=("docker","compose.yaml"), # yaml contains the default for a sandbox
        solver=[react(tools=[treesitter_server_sandbox, fs_server])],
        
        scorer=[
            includes(),
        ],

        # solver=[use_tools=patrick_fixer])
    )

results = eval(basic_mcp,log_level="info",display="full",fail_on_error=True,time_limit=60, sandbox_cleanup=True)
p4 = pretty_results(results,show_events=False)
print(p4)